# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-22-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-22-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-23 04:23:40,33.93911,67.709953,39096,1445,32576,5075.0,Afghanistan,100.430686,3.696030
1,NaN,NaN,NaN,Albania,2020-09-23 04:23:40,41.15330,20.168300,12666,367,7042,5257.0,Albania,440.127875,2.897521
2,NaN,NaN,NaN,Algeria,2020-09-23 04:23:40,28.03390,1.659600,50214,1689,35307,13218.0,Algeria,114.510389,3.363604
3,NaN,NaN,NaN,Andorra,2020-09-23 04:23:40,42.50630,1.521800,1681,53,1199,429.0,Andorra,2175.629328,3.152885
4,NaN,NaN,NaN,Angola,2020-09-23 04:23:40,-11.20270,17.873900,4236,155,1462,2619.0,Angola,12.888594,3.659112


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38716,38772,38815,38855,38872,38883,38919,39044,39074,39096
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,11353,11520,11672,11816,11948,12073,12226,12385,12535,12666
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,48254,48496,48734,48966,49194,49413,49623,49826,50023,50214


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1420,1425,1426,1436,1436,1437,1437,1441,1444,1445
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334,338,340,343,347,353,358,362,364,367
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1612,1620,1632,1645,1654,1659,1665,1672,1679,1689


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,31638,32073,32098,32503,32505,32576,32576,32576,32576,32576
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6569,6615,6668,6733,6788,6831,6888,6940,6995,7042
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,34037,34204,34385,34517,34675,34818,34923,35047,35180,35307


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1442,1447,1463,1619,1624,1664,1673,1690,1691,1714
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4781,4800,4812,5003,5021,5033,5047,5061,5087,5124
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,622,626,629,809,809,824,830,835,838,848


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,23,23,23,24,24,24,24,24,24,25
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,43,43,43,47,48,48,49,49,49,49
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
632,1001.0,Autauga,Alabama,US,2020-09-23 04:23:40,32.539527,-86.644082,1714,25,0,1689.0,"Autauga, Alabama, US",3067.890959,1.458576
669,1075.0,Lamar,Alabama,US,2020-09-23 04:23:40,33.779950,-88.096680,355,2,0,353.0,"Lamar, Alabama, US",2571.532054,0.563380
670,1077.0,Lauderdale,Alabama,US,2020-09-23 04:23:40,34.901719,-87.656247,1662,33,0,1629.0,"Lauderdale, Alabama, US",1792.319555,1.985560


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,6896218,200786,2646959
India,5562663,88935,4497867
Brazil,4591364,138105,4016831


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,6896218,200786,2646959,4048473,2020-09-23 04:23:40,37.923092,-91.345036
India,5562663,88935,4497867,975861,2020-09-23 04:23:40,23.088275,81.806127
Brazil,4591364,138105,4016831,436428,2020-09-23 04:23:40,-12.669522,-48.480493
Russia,1111157,19575,914923,176659,2020-09-23 04:23:40,54.546312,62.120860
Colombia,777537,24397,640900,112240,2020-09-23 04:23:40,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,793065,15189,0
Texas,741663,15229,0
Florida,687909,13416,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,793065,15189,0,777876,2020-09-23 04:23:40,37.843962,-120.728594
Texas,741663,15229,0,726434,2020-09-23 04:23:40,31.660643,-98.653069
Florida,687909,13416,0,674493,2020-09-23 04:23:40,28.940755,-82.700744
New York,451227,33090,0,418137,2020-09-23 04:23:40,42.544151,-75.474183
Georgia,308294,6673,0,301621,2020-09-23 04:23:40,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,262133,6401,0
Florida,Miami-Dade,167515,3085,0
Arizona,Maricopa,140314,3275,0
Illinois,Cook,140078,5172,0
Texas,Harris,138473,2518,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,262133,6401,0,255732,2020-09-23 04:23:40,34.308284,-118.228241,6037.0
Florida,Miami-Dade,167515,3085,0,164430,2020-09-23 04:23:40,25.611236,-80.551706,12086.0
Arizona,Maricopa,140314,3275,0,137039,2020-09-23 04:23:40,33.348359,-112.491815,4013.0
Illinois,Cook,140078,5172,0,134906,2020-09-23 04:23:40,41.841448,-87.816588,17031.0
Texas,Harris,138473,2518,0,135955,2020-09-23 04:23:40,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,39044,12385,49826,1564,3991,96,631365,47431,26912,38095,...,396744,1917,51640,66656,1068,35686,10,2026,14131,7683
2020-09-21,39074,12535,50023,1681,4117,96,640147,47552,26942,38658,...,401122,1927,52070,67443,1068,36151,10,2028,14175,7683
2020-09-22,39096,12666,50214,1681,4236,96,652174,47667,26972,39303,...,406058,1934,52685,68453,1068,36580,10,2028,14389,7711


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,1441,362,1672,53,152,3,13053,930,851,766,...,41866,46,433,547,35,262,1,586,330,225
2020-09-21,1444,364,1679,53,154,3,13482,936,854,767,...,41877,46,437,555,35,265,1,586,331,225
2020-09-22,1445,367,1689,53,155,3,13952,938,859,771,...,41914,46,442,564,35,269,1,586,331,226


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,32576,6940,35047,1164,1445,92,488231,42608,24061,29229,...,2228,1621,47932,56096,942,23700,8,1227,13365,5924
2020-09-21,32576,6995,35180,1199,1449,92,508563,42637,24155,29516,...,2228,1634,48369,56726,947,24428,8,1235,13629,5924
2020-09-22,32576,7042,35307,1199,1462,92,517228,42676,24329,30312,...,2238,1645,49067,57774,957,25469,8,1240,13629,5979


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1442,1447,1463,1619,1624,1664,1673,1690,1691,1714
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,292,295,299,327,330,337,345,346,353,355
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1527,1530,1539,1590,1592,1616,1641,1646,1653,1662
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,532,534,546,573,576,578,582,585,592,603
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,3613,3622,3624,5570,5582,5603,5705,5713,5751,5765


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-20,144962,6822,0,214021,75723,786168,64837,55527,19566,49,...,183514,713007,63772,1715,1269,140395,82548,14062,101227,4872
2020-09-21,145780,6892,0,214251,76364,790096,65379,56024,19667,49,...,184409,734778,64394,1719,1269,141022,82730,14181,102498,4944
2020-09-22,146584,6936,0,214846,76981,793065,66033,56160,19761,49,...,185148,741663,65044,1721,1278,141884,83193,14394,104170,5016


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-20        1690    5061     835  632   1551     587    909    3338   
2020-09-21        1691    5087     838  636   1560     591    911    3374   
2020-09-22        1714    5124     848  635   1573     593    911    3390   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-20         1130      565  ...     202     20      242       73   
2020-09-21         1132      569  ...     205     20      253       71   
2020-09-22         1140      575  ...     210     21      254       79   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-20            323   498   334          1      113     23  
2020-09-21            323   504   335          0      113     23  
2020-09-22            323   506   339          0      113     26  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-20,2437,45,0,5477,1181,15016,2014,4492,621,0,...,2218,15088,440,58,19,3013,2037,314,1242,49
2020-09-21,2439,45,0,5478,1197,15056,2018,4495,627,0,...,2233,15127,441,58,19,3019,2049,315,1244,49
2020-09-22,2457,45,0,5498,1209,15189,2025,4496,628,0,...,2261,15229,443,58,19,3058,2070,320,1251,49


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-20          24      49       7   10     14      14     38      39   
2020-09-21          24      49       7   10     14      14     38      39   
2020-09-22          25      49       7   10     14      14     38      40   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-20           42       16  ...       2      1        4        1   
2020-09-21           42       16  ...       2      1        4        1   
2020-09-22           42       16  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-20              2     1     2          0        6      0  
2020-09-21              2     1     2          0        6      0  
2020-09-22              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,0.003212,0.013005,0.004091,0.000000,0.023071,0.0,0.013534,0.005874,0.000520,0.016571,...,0.009925,0.006828,0.012708,0.010720,0.0,0.019513,0.0,0.000000,0.004335,0.001434
2020-09-21,0.000768,0.012111,0.003954,0.074808,0.031571,0.0,0.013910,0.002551,0.001115,0.014779,...,0.011035,0.005216,0.008327,0.011807,0.0,0.013030,0.0,0.000987,0.003114,0.000000
2020-09-22,0.000563,0.010451,0.003818,0.000000,0.028905,0.0,0.018788,0.002418,0.001114,0.016685,...,0.012305,0.003633,0.011811,0.014976,0.0,0.011867,0.0,0.000000,0.015097,0.003644


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,0.002784,0.011173,0.004204,0.0,0.034014,0.0,0.019845,0.002155,0.002356,0.001307,...,0.000430,0.0,0.014052,0.014842,0.0,0.035573,0.0,0.001709,0.00000,0.000000
2020-09-21,0.002082,0.005525,0.004187,0.0,0.013158,0.0,0.032866,0.006452,0.003525,0.001305,...,0.000263,0.0,0.009238,0.014625,0.0,0.011450,0.0,0.000000,0.00303,0.000000
2020-09-22,0.000693,0.008242,0.005956,0.0,0.006494,0.0,0.034861,0.002137,0.005855,0.005215,...,0.000884,0.0,0.011442,0.016216,0.0,0.015094,0.0,0.000000,0.00000,0.004444


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,0.0,0.007549,0.003551,0.000000,0.000000,0.0,0.021239,0.001340,0.003964,0.009254,...,0.000449,0.005583,0.013983,0.017061,0.000000,0.010833,0.0,0.004914,0.000000,0.001691
2020-09-21,0.0,0.007925,0.003795,0.030069,0.002768,0.0,0.041644,0.000681,0.003907,0.009819,...,0.000000,0.008020,0.009117,0.011231,0.005308,0.030717,0.0,0.006520,0.019753,0.000000
2020-09-22,0.0,0.006719,0.003610,0.000000,0.008972,0.0,0.017038,0.000915,0.007203,0.026968,...,0.004488,0.006732,0.014431,0.018475,0.010560,0.042615,0.0,0.004049,0.000000,0.009284


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-20,0.005535,0.013821,NaN,0.002201,0.007491,0.003645,0.007787,0.000000,0.006016,0.0,...,0.011436,0.007157,0.014638,0.002924,0.009547,0.006084,0.004246,0.012456,0.016723,0.019034
2020-09-21,0.005643,0.010261,NaN,0.001075,0.008465,0.004996,0.008359,0.008951,0.005162,0.0,...,0.004877,0.030534,0.009753,0.002332,0.000000,0.004466,0.002205,0.008463,0.012556,0.014778
2020-09-22,0.005515,0.006384,NaN,0.002777,0.008080,0.003758,0.010003,0.002428,0.004780,0.0,...,0.004007,0.009370,0.010094,0.001163,0.007092,0.006113,0.005597,0.015020,0.016313,0.014563


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-20      0.010161  0.002774  0.006024  0.006369  0.005837  0.003419   
2020-09-21      0.000592  0.005137  0.003593  0.006329  0.005803  0.006814   
2020-09-22      0.013601  0.007273  0.011933 -0.001572  0.008333  0.003384   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-20      0.001101  0.005422  0.006233  0.018018  ...  0.000000   
2020-09-21      0.002200  0.010785  0.001770  0.007080  ...  0.014851   
2020-09-22      0.000000  0.004742  0.007067  0.010545  ...  0.024390   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-20      0.052632  0.034188  0.028169   0.003106  0.010142  0.012121   
2020-09-21      0.000000  0.045455 -0.027397   0.000000  0.012048  0.002994   
2020-09-22      0.050000  0.003953  0.112676   0.000000  0.003968  0.011940   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2020-09-20            0.0      0.0  0.000000  
2020-09-21           -1.0      0.0  0.000000  
2020-09-22            NaN      0.0  0.130435  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-20,0.000000,0.0,NaN,0.001829,0.000000,-0.000133,0.000497,0.000000,0.000000,NaN,...,0.000903,0.002458,0.000000,0.0,0.0,0.008367,0.000000,0.009646,0.000806,0.0
2020-09-21,0.000821,0.0,NaN,0.000183,0.013548,0.002664,0.001986,0.000668,0.009662,NaN,...,0.006763,0.002585,0.002273,0.0,0.0,0.001991,0.005891,0.003185,0.001610,0.0
2020-09-22,0.007380,0.0,NaN,0.003651,0.010025,0.008834,0.003469,0.000222,0.001595,NaN,...,0.012539,0.006743,0.004535,0.0,0.0,0.012918,0.010249,0.015873,0.005627,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-09-20      0.000000     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   
2020-09-21      0.000000     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   
2020-09-22      0.041667     0.0     0.0  0.0    0.0     0.0    0.0  0.025641   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-20          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-09-21          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-09-22          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-20            0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-21            0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-22            0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,0.001972,0.012500,0.004262,0.008394,0.020643,0.002632,0.015623,0.005549,0.000695,0.019327,...,0.010371,0.007603,0.013147,0.011833,0.000425,0.019328,4.487742e-29,0.000551,0.004481,0.002270
2020-09-21,0.001370,0.012306,0.004108,0.041601,0.026107,0.001316,0.014766,0.004050,0.000905,0.017053,...,0.010703,0.006410,0.010737,0.011820,0.000212,0.016179,2.243871e-29,0.000769,0.003798,0.001135
2020-09-22,0.000967,0.011378,0.003963,0.020801,0.027506,0.000658,0.016777,0.003234,0.001009,0.016869,...,0.011504,0.005021,0.011274,0.013398,0.000106,0.014023,1.121936e-29,0.000385,0.009447,0.002390


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,0.001737,0.012560,0.004165,1.749028e-14,0.021516,1.379147e-48,0.017745,0.002063,0.004601,0.002210,...,0.000517,0.001389,0.015412,0.016267,1.517116e-07,0.026728,0.0,0.001083,0.002560,0.001135
2020-09-21,0.001910,0.009042,0.004176,8.745141e-15,0.017337,6.895733e-49,0.025305,0.004258,0.004063,0.001758,...,0.000390,0.000695,0.012325,0.015446,7.585579e-08,0.019089,0.0,0.000542,0.002795,0.000568
2020-09-22,0.001301,0.008642,0.005066,4.372571e-15,0.011915,3.447866e-49,0.030083,0.003197,0.004959,0.003486,...,0.000637,0.000347,0.011883,0.015831,3.792790e-08,0.017092,0.0,0.000271,0.001398,0.002506


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-20,0.000852,0.007733,0.003610,0.016673,0.005824,0.001374,0.021913,0.002613,0.003615,0.012485,...,0.003735,0.007363,0.013967,0.019496,0.001062,0.012002,2.629536e-31,0.002585,0.006614,0.003767
2020-09-21,0.000426,0.007829,0.003702,0.023371,0.004296,0.000687,0.031779,0.001647,0.003761,0.011152,...,0.001867,0.007691,0.011542,0.015364,0.003185,0.021360,1.314768e-31,0.004552,0.013184,0.001884
2020-09-22,0.000213,0.007274,0.003656,0.011686,0.006634,0.000343,0.024408,0.001281,0.005482,0.019060,...,0.003178,0.007212,0.012986,0.016919,0.006872,0.031987,6.573841e-32,0.004300,0.006592,0.005584


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-20,0.006701,0.013706,NaN,0.003260,0.010385,0.004326,0.008423,0.000772,0.005077,8.677267e-58,...,0.009627,0.006189,0.015634,0.002376,0.007193,0.006777,0.005142,0.014314,0.020731,0.016260
2020-09-21,0.006172,0.011983,NaN,0.002167,0.009425,0.004661,0.008391,0.004861,0.005119,4.338633e-58,...,0.007252,0.018361,0.012694,0.002354,0.003596,0.005622,0.003673,0.011388,0.016644,0.015519
2020-09-22,0.005844,0.009184,NaN,0.002472,0.008752,0.004210,0.009197,0.003644,0.004949,2.169317e-58,...,0.005630,0.013866,0.011394,0.001759,0.005344,0.005867,0.004635,0.013204,0.016478,0.015041


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-20      0.013294  0.003969  0.016237  0.009512  0.017802  0.003375   
2020-09-21      0.006943  0.004553  0.009915  0.007920  0.011803  0.005095   
2020-09-22      0.010272  0.005913  0.010924  0.003174  0.010068  0.004239   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-20      0.002952  0.014831  0.014424  0.026423  ...  0.004447   
2020-09-21      0.002576  0.012808  0.008097  0.016751  ...  0.009649   
2020-09-22      0.001288  0.008775  0.007582  0.013648  ...  0.017020   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-20      0.062706  0.025932  0.029123   0.002888  0.008591  0.010271   
2020-09-21      0.031353  0.035693  0.000863   0.001444  0.010320  0.006632   
2020-09-22      0.040676  0.019823  0.056769   0.000722  0.007144  0.009286   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-20      -0.038837  0.001679  0.000200  
2020-09-21      -0.571428  0.000840  0.000100  
2020-09-22      -0.571428  0.000420  0.065267  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-20,0.002893,0.002910,NaN,0.003305,0.005815,0.003717,0.001214,0.000149,0.000723,NaN,...,0.005705,0.004877,0.001859,2.512958e-16,0.000118,0.010785,0.000924,0.017876,0.002317,0.004703
2020-09-21,0.001857,0.001455,NaN,0.001744,0.009681,0.003191,0.001600,0.000408,0.005192,NaN,...,0.006234,0.003731,0.002066,1.256479e-16,0.000059,0.006388,0.003408,0.010530,0.001963,0.002351
2020-09-22,0.004618,0.000728,NaN,0.002697,0.009853,0.006012,0.002534,0.000315,0.003394,NaN,...,0.009387,0.005237,0.003301,6.282395e-17,0.000029,0.009653,0.006828,0.013202,0.003795,0.001176


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-09-20      0.001359  0.009469  1.949353e-11  0.029513  0.009779   
2020-09-21      0.000679  0.004734  9.746765e-12  0.014757  0.004889   
2020-09-22      0.021173  0.002367  4.873383e-12  0.007378  0.002445   

Province_State                                                    ...  \
Admin2           Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-09-20      0.002404  0.000845  0.008701  0.001563  0.015530  ...   
2020-09-21      0.001202  0.000422  0.004351  0.000781  0.007765  ...   
2020-09-22      0.000601  0.000211  0.014996  0.000391  0.003883  ...   

Province_State   Wyoming                                                   \
Admin2              Park Platte  Sheridan Sublette Sweetwater Teton Uinta   
2020-09-20      0.062500    0.0  0.093750      0.0        0.0   0.0   0.0   
2020-09-21      0.031250    0.0  0.046875      0.0        0.0   0.0   0.0   
2020-09-22      0.015625    0.0  0.023438      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-20      -0.999969      0.0    NaN  
2020-09-21      -0.999969      0.0    NaN  
2020-09-22      -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200922,AK,7886,423305,NaN,431191,43.0,NaN,NaN,NaN,...,431191,0,0,24135b606b72e9c822c662be4c07b364d622accd,0,0,0,0,0,NaN
1,20200922,AL,146584,933565,NaN,1065553,796.0,16604.0,NaN,1659.0,...,1080149,18,117,536eaa13e29a33e56ab8f49589c94d7bd8cd3164,0,0,0,0,0,NaN
2,20200922,AR,76981,823225,NaN,897997,453.0,5055.0,234.0,NaN,...,900206,12,69,23ebdd8ed12a95784875ca7899fc5d37df6a2fb3,0,0,0,0,0,NaN
3,20200922,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,...,1571,0,0,1fbce7d9c6b9b287e0974b92a27198573fb8ecdf,0,0,0,0,0,NaN
4,20200922,AZ,214846,1185808,NaN,1400654,527.0,21900.0,122.0,NaN,...,1400654,20,22,b86f1576190f1bb4bf95fd99bab7f75a41235b33,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200922,AK,7886.0,423305.0,NaN,431191.0,43.0,NaN,NaN,NaN,...,431191,0,0,24135b606b72e9c822c662be4c07b364d622accd,0,0,0,0,0,NaN
1,20200922,AL,146584.0,933565.0,NaN,1065553.0,796.0,16604.0,NaN,1659.0,...,1080149,18,117,536eaa13e29a33e56ab8f49589c94d7bd8cd3164,0,0,0,0,0,NaN
2,20200922,AR,76981.0,823225.0,NaN,897997.0,453.0,5055.0,234.0,NaN,...,900206,12,69,23ebdd8ed12a95784875ca7899fc5d37df6a2fb3,0,0,0,0,0,NaN
3,20200922,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,1fbce7d9c6b9b287e0974b92a27198573fb8ecdf,0,0,0,0,0,NaN
4,20200922,AZ,214846.0,1185808.0,NaN,1400654.0,527.0,21900.0,122.0,NaN,...,1400654,20,22,b86f1576190f1bb4bf95fd99bab7f75a41235b33,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-22,AK,7886,423305,NaN,431191,43.0,NaN,NaN,NaN,14.0,...,431191,0,0,24135b606b72e9c822c662be4c07b364d622accd,0,0,0,0,0,NaN
2020-09-22,AL,146584,933565,NaN,1065553,796.0,16604.0,NaN,1659.0,NaN,...,1080149,18,117,536eaa13e29a33e56ab8f49589c94d7bd8cd3164,0,0,0,0,0,NaN
2020-09-22,AR,76981,823225,NaN,897997,453.0,5055.0,234.0,NaN,87.0,...,900206,12,69,23ebdd8ed12a95784875ca7899fc5d37df6a2fb3,0,0,0,0,0,NaN
2020-09-22,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,1fbce7d9c6b9b287e0974b92a27198573fb8ecdf,0,0,0,0,0,NaN
2020-09-22,AZ,214846,1185808,NaN,1400654,527.0,21900.0,122.0,NaN,47.0,...,1400654,20,22,b86f1576190f1bb4bf95fd99bab7f75a41235b33,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-22,AK,7886.0,423305.0,NaN,431191.0,43.0,NaN,NaN,NaN,14.0,...,431191,0,0,24135b606b72e9c822c662be4c07b364d622accd,0,0,0,0,0,NaN
2020-09-22,AL,146584.0,933565.0,NaN,1065553.0,796.0,16604.0,NaN,1659.0,NaN,...,1080149,18,117,536eaa13e29a33e56ab8f49589c94d7bd8cd3164,0,0,0,0,0,NaN
2020-09-22,AR,76981.0,823225.0,NaN,897997.0,453.0,5055.0,234.0,NaN,87.0,...,900206,12,69,23ebdd8ed12a95784875ca7899fc5d37df6a2fb3,0,0,0,0,0,NaN
2020-09-22,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,1fbce7d9c6b9b287e0974b92a27198573fb8ecdf,0,0,0,0,0,NaN
2020-09-22,AZ,214846.0,1185808.0,NaN,1400654.0,527.0,21900.0,122.0,NaN,47.0,...,1400654,20,22,b86f1576190f1bb4bf95fd99bab7f75a41235b33,0,0,0,0,0,NaN
